# EECS 491: Probabilistic Graphical Models Assignment 1
**David Fan**

2/28/18

# Exercise 1

## Problem Setup

Let us say that we are interested in being able to determine whether or not I will be late for class ($L$) and whether a classmate will be late for class ($C$). Whether or not I will be late for class is effected by two things:
* If I oversleep ($O$)
* If the bus is late ($B$)

The bus being late also impacts whether or not a classmate will be late for class. Whether or not I oversleep is effected by whether or not my alarm goes off ($A$).

From experience I have a prior belief that my alarm will go off with probability .96. I have a prior belief that given that my alarm goes off, I will oversleep with probability .08, and if my alarm doesn't go off, I will oversleep with probability .98. I also have a prior belief that my oversleeping does not have any effect on whether my classmate is late for class.

Also from my experience, I have a prior belief that the bus will be late ($B$) with probability .2. I estimate that if the bus is late and I oversleep, I will be late with probability .96. If the bus is late and I don't oversleep, I will be late with probability .78. If the bus isn't late and I oversleep, I'll be late with probability .8. If the bus isn't late and I don't oversleep, I'll be late with probability .03. If the bus is late, I estimate that my classmate will be late with probability .8. If the bus isn't late, I estimate that my classmate will be late with probability .1 These probabilities are summarized in the tables below:

$$
\begin{align}
P(A) & = 0.96\\
P(B) & = .2
\end{align}
$$

|           | A = false | A = true |
| :--------- | -------------------- | ------------------- |
| O = false | 0.02 | 0.92 |
| O = true  | 0.98 | 0.08 |

|           | B = false, O = false | B = false, O = true | B = true, O = false | B = true, O = true |
| :--------- | -------------------- | ------------------- | ------------------- | ------------------ |
| L = false | 0.97 | 0.2 | 0.22 | 0.04 |
| L = true  | 0.03 | 0.8 | 0.78 | 0.96 |

|           | B = false | B = true |
| :--------- | -------------------- | ------------------- |
| C = false | 0.9 | 0.2 |
| C = true  | 0.1 | 0.8 |

The model can be presented in the following graph:
<center>
<img src="imgs/ex1_diagram.jpeg" width='40%'/>
</center>

## Programatic Model Setup

In [2]:
from pgmpy.models import BayesianModel as bysmodel
from pgmpy.factors.discrete import TabularCPD as tcpd

In [3]:
model = bysmodel([['A','O'],['O','L'],['B','L'],['B','C']])

In [4]:
priorA = tcpd(variable='A', variable_card=2, values=[[0.04, 0.96]])
priorB = tcpd(variable='B', variable_card=2, values=[[.8,.2]])

In [5]:
cpdO = tcpd(variable='O', variable_card=2, 
            evidence=['A'], evidence_card=[2],
            values=[[0.02, 0.92], 
                    [0.98, 0.08]]
           )
cpdL = tcpd(variable='L', variable_card=2,
            evidence=['O', 'B'], evidence_card=[2,2],
            values=[[.97, .2, .22, .04],
                    [.03, .8, .78, .96]]
           )
cpdC = tcpd(variable='C', variable_card=2,
            evidence=['B'], evidence_card=[2],
            values=[[.9, .2],
                    [.1, .8]])

In [6]:
model.add_cpds(priorA, priorB, cpdO, cpdL, cpdC)

Now we can check if our model is valid:

In [7]:
model.check_model()

True

Our model is now set up in pgmpy

In [9]:
print(model.get_cpds('O'))
print(model.get_cpds('L'))
print(model.get_cpds('C'))

╒═════╤══════╤══════╕
│ A   │ A_0  │ A_1  │
├─────┼──────┼──────┤
│ O_0 │ 0.02 │ 0.92 │
├─────┼──────┼──────┤
│ O_1 │ 0.98 │ 0.08 │
╘═════╧══════╧══════╛
╒═════╤══════╤═════╤══════╤══════╕
│ O   │ O_0  │ O_0 │ O_1  │ O_1  │
├─────┼──────┼─────┼──────┼──────┤
│ B   │ B_0  │ B_1 │ B_0  │ B_1  │
├─────┼──────┼─────┼──────┼──────┤
│ L_0 │ 0.97 │ 0.2 │ 0.22 │ 0.04 │
├─────┼──────┼─────┼──────┼──────┤
│ L_1 │ 0.03 │ 0.8 │ 0.78 │ 0.96 │
╘═════╧══════╧═════╧══════╧══════╛
╒═════╤═════╤═════╕
│ B   │ B_0 │ B_1 │
├─────┼─────┼─────┤
│ C_0 │ 0.9 │ 0.2 │
├─────┼─────┼─────┤
│ C_1 │ 0.1 │ 0.8 │
╘═════╧═════╧═════╛


In [ ]:
from pgmpy.inference import VariableElimination as proc

In [ ]:
infer = proc(model)
print(infer.query(['A'], evidence={'L' : 1, 'C' : 0})['A'])